# 0. Initialisation

## 0.1 Imports

In [4]:
import pandas as pd
import numpy as np
from collections import Counter
import chardet
import editdistance #
import jellyfish
import re
from tqdm import tqdm
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#DATA_PATH = r"C:\Users\Louis\Documents\University\Masters\A23\NLP\Devoirs\data\hw2"
DATA_PATH = r"C:\Users\barka\Desktop\NLP"

## 0.2 Functions

In [5]:
def detect_encoding(file_path):

    with open(file_path, 'rb') as file:
        rawdata = file.read()
    result = chardet.detect(rawdata)
    return result['encoding']

def get_word_counter(file_path, encoding):
    word_counter = Counter()

    with open(file_path, 'r', encoding=encoding) as file:
        for line in file:
            parts = line.split()
            if len(parts) >= 2:
                word = parts[1]
                count = int(parts[0]) 
                word_counter[word.strip()] = count

    return word_counter


def build_unigram_model(word_counter):
    unigram_model = {}

    total_words = sum(word_counter.values())

    for word, count in word_counter.items():
        probability = count / total_words

        unigram_model[word] = probability

    return unigram_model

## 0.3 Check data quality
Make sure Python is reading the lines correctly. And it does!

In [6]:
# Check ypos
typos_file = open(DATA_PATH + r"\typo-0.2.txt")

for i, row in enumerate(typos_file):
    #print(row)
    pass

print(f"There are {i+1} rows in the file. There should be 1000. Correct number? {i+1==1000}")

# Check vocabulary

encoding = detect_encoding(DATA_PATH + r"\voc-1bwc.txt") #check encoding
print(f"Encoding : {encoding}")

voc = open(DATA_PATH + r"\voc-1bwc.txt", encoding=encoding)

for i, row in enumerate(voc):
    #print(row)
    pass

print(f"There are {i+1} rows in the file. There should be 201 315. Correct number? {i+1==201315}")

There are 1000 rows in the file. There should be 1000. Correct number? True
Encoding : utf-8
There are 202971 rows in the file. There should be 201 315. Correct number? False


How many words in the vocabulary?

In [7]:
vocab = get_word_counter(DATA_PATH + r"\voc-1bwc.txt", encoding)
print(len(vocab)) # just to verify

202971


In [8]:
vocab["the"]

41640816

In [9]:
unigram_model = build_unigram_model(vocab)

In [10]:
unigram_model["the"]

0.06258941042979899

# 1. Implementing various distances

### 1.1 Generic Distance function

Works with any function that takes two words as an input. 

In [11]:
'''This function applies a generic function over an entire vocabulary to compare the distance between a given word
and every word inside the vocabulary. It then returns the n_neighbors most similar words.

Parameters:
    word: The word to find neighbors to.
    vocabulary: A list (or list-like) of the vocabulary.
    func: The distance function to apply without arguments (without parentheses).
    minimum: Do we want the minimum distance? Boolean. True by default, will yield the minimum distance. If False, the function will yield the maximum distance. 
    n_neighbors: The number of most similar words to return.

Returns:
    vocabulary.head: A dataframe containing the n_neighbors most similar words to the input word, with the distances. 
'''
def generic_distance_correction(word, vocabulary, func, minimum = True, n_neighbors=1):
    vocabulary = pd.DataFrame(vocabulary, columns=["words"])

    def calculateDistance(series_word):
        return func(series_word, word)
    
    distances = vocabulary["words"].apply(calculateDistance)
    
    vocabulary["distance"] = distances

    vocabulary = vocabulary.sort_values("distance", ascending=minimum)

    return vocabulary.head(n_neighbors)

### Weighted Functions

In [12]:
def edDistance_unigram_correction(word, vocabulary, n_neighbors=1):
    vocabulary = pd.DataFrame(vocabulary, columns=["words"])

    def calculateDistance(series_word):
        return editdistance.eval(series_word, word)
    
    def unigram_weighing(word):
        return unigram_model[word]

    distances = vocabulary["words"].apply(calculateDistance)
    vocabulary["distance"] = distances
    vocabulary["weight"] = vocabulary["words"].apply(unigram_weighing)

    vocabulary = vocabulary.sort_values(["distance", "weight"], ascending=[True, False])

    return vocabulary.head(n_neighbors)

def edDistance_soundex_correction(word, vocabulary, n_neighbors=1):
    vocabulary = pd.DataFrame(vocabulary, columns=["words"])

    def calculateDistance(series_word):
        return editdistance.eval(series_word, word)
    
    distances = vocabulary["words"].apply(calculateDistance)
    vocabulary["distance"] = distances
    vocabulary = vocabulary.sort_values("distance", ascending=True)
    
    # Then, out of the head, sort by soundex
    top_words = vocabulary.head(n_neighbors)
    soundex = []
    word_sdx = jellyfish.soundex(word)
    for word in top_words["words"]:
        corr_sdx = jellyfish.soundex(word)
        sdx_ed_distance = editdistance.eval(word_sdx, corr_sdx)
        soundex.append(sdx_ed_distance)
    top_words["soundex"] = soundex
    top_words = top_words.sort_values(["distance", "soundex"], ascending=[True, True])

    return top_words

In [13]:
def numeric_unigram_correction(word, vocabulary, func, n_neighbors=1):
    vocabulary = pd.DataFrame(vocabulary, columns=["words"])

    def calculateDistance(series_word):
        return func(series_word, word)
    
    def unigram_weighing(word):
        return unigram_model[word]

    distances = vocabulary["words"].apply(calculateDistance)
    vocabulary["distance"] = distances

    # Sort the values using the weight and only keep the head
    top_words = vocabulary.sort_values("distance", ascending=False).head(n_neighbors)

    # Make the weights relative to each other, using only the top words
    top_words["weight"] = top_words["words"].apply(unigram_weighing)
    top_words["weight"] = top_words["weight"] / top_words["weight"].sum()

    # Weight the distance
    top_words["wgt_distance"] = top_words["distance"] * top_words["weight"]

    # Sort the top words using the weighted distance
    top_words = top_words.sort_values("wgt_distance", ascending=False)

    return top_words

def numeric_soundex_correction(word, vocabulary, func, n_neighbors=1):
    vocabulary = pd.DataFrame(vocabulary, columns=["words"])

    def calculateDistance(series_word):
        return func(series_word, word)
    
    distances = vocabulary["words"].apply(calculateDistance)
    vocabulary["distance"] = distances
    vocabulary = vocabulary.sort_values("distance", ascending=False)
    
    # Then, out of the head, sort by soundex
    top_words = vocabulary.head(n_neighbors)
    soundex = []
    word_sdx = jellyfish.soundex(word)
    for word in top_words["words"]:
        corr_sdx = jellyfish.soundex(word)
        sdx_ed_distance = editdistance.eval(word_sdx, corr_sdx)
        soundex.append(sdx_ed_distance)
    top_words["soundex"] = 1 - soundex / np.sum(soundex)
    top_words["wgt_distance"] = top_words["distance"] * top_words["soundex"]
    
    top_words = top_words.sort_values("wgt_distance", ascending=False)

    return top_words

### 1.2. Testing the correction methods

In [14]:
generic_distance_correction('speling', vocab.keys(), editdistance.eval, True, 5) #Edit distance

,words,distance
172484,sperling,1
191321,spelling,1
179101,spewing,1
123329,spaying,2
187893,seeming,2


In [15]:
edDistance_unigram_correction("speling", vocab.keys(), 5)

,words,distance,weight
191321,spelling,1,0.000005
179101,spewing,1,0.000002
172484,sperling,1,0.000001
202398,spending,2,0.000200
202060,opening,2,0.000131


In [16]:
edDistance_soundex_correction("speling", vocab.keys(), 5)

C:\Users\barka\AppData\Local\Temp\ipykernel_18524\3376419266.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_words["soundex"] = soundex


,words,distance,soundex
191321,spelling,1,0
172484,sperling,1,2
179101,spewing,1,2
123329,spaying,2,2
187893,seeming,2,3


In [17]:
generic_distance_correction('speling', vocab.keys(), jellyfish.jaro_similarity, False, 5)

,words,distance
191321,spelling,0.958333
172484,sperling,0.958333
168457,spellings,0.925926
197912,sleeping,0.910714
187411,selig,0.904762


In [18]:
numeric_unigram_correction("speling", vocab.keys(), jellyfish.jaro_similarity, 5)

,words,distance,weight,wgt_distance
197912,sleeping,0.910714,0.641903,0.584590
191321,spelling,0.958333,0.181914,0.174334
187411,selig,0.904762,0.113302,0.102511
172484,sperling,0.958333,0.035068,0.033607
168457,spellings,0.925926,0.027813,0.025753


In [19]:
numeric_soundex_correction("speling", vocab.keys(), jellyfish.jaro_similarity, 5)

C:\Users\barka\AppData\Local\Temp\ipykernel_18524\1142548253.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_words["soundex"] = 1 - soundex / np.sum(soundex)
C:\Users\barka\AppData\Local\Temp\ipykernel_18524\1142548253.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_words["wgt_distance"] = top_words["distance"] * top_words["soundex"]


,words,distance,soundex,wgt_distance
191321,spelling,0.958333,1.000000,0.958333
168457,spellings,0.925926,1.000000,0.925926
172484,sperling,0.958333,0.714286,0.684524
197912,sleeping,0.910714,0.714286,0.650510
187411,selig,0.904762,0.571429,0.517007


In [20]:
generic_distance_correction('speling', vocab.keys(), jellyfish.jaro_winkler_similarity, False, 5)

,words,distance
191321,spelling,0.975000
172484,sperling,0.970833
168457,spellings,0.955556
179101,spewing,0.933333
197912,sleeping,0.919643


In [21]:
numeric_unigram_correction("speling", vocab.keys(), jellyfish.jaro_winkler_similarity, 5)

,words,distance,weight,wgt_distance
197912,sleeping,0.919643,0.681877,0.627083
191321,spelling,0.975000,0.193242,0.188411
179101,spewing,0.933333,0.058084,0.054212
172484,sperling,0.970833,0.037252,0.036166
168457,spellings,0.955556,0.029545,0.028232


In [22]:
numeric_soundex_correction("speling", vocab.keys(), jellyfish.jaro_winkler_similarity, 5)

C:\Users\barka\AppData\Local\Temp\ipykernel_18524\1142548253.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_words["soundex"] = 1 - soundex / np.sum(soundex)
C:\Users\barka\AppData\Local\Temp\ipykernel_18524\1142548253.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_words["wgt_distance"] = top_words["distance"] * top_words["soundex"]


,words,distance,soundex,wgt_distance
191321,spelling,0.975000,1.000000,0.975000
168457,spellings,0.955556,1.000000,0.955556
172484,sperling,0.970833,0.666667,0.647222
179101,spewing,0.933333,0.666667,0.622222
197912,sleeping,0.919643,0.666667,0.613095


## 1.3. Running the correction methods on the data

### 1.3.1 - Get the typos:

In [23]:
with open(DATA_PATH + r"\typo-0.2.txt", "r", encoding=encoding) as file:
    text = file.read()

typo_pattern = r'<typo orig="([^"]+)">([^<]+)</typo>'
typos = re.findall(typo_pattern, text)
typos = pd.DataFrame(typos, columns=["Word", "Typo"])


typos.tail(10)

,Word,Typo
3173,defy,deefy
3174,translation,translatmion
3175,to,tho
3176,But,ut
3177,in,ini
3178,keep,kpeep
3179,game,gme
3180,winds,wantagh
3181,this,tsi
3182,of,o


In [24]:
s1 = set(typos["Word"])
s2 = set(vocab.keys())
inter = s2.intersection(s1)

print(f"There are {len(typos['Word'].unique())} unique original words and {len(pd.Series(vocab.keys()).unique())} unique words in the vocabulary. Out of these two, {len(inter)} intersect.")


There are 1601 unique original words and 202971 unique words in the vocabulary. Out of these two, 1279 intersect.


### 1.3.2 - Apply correction methods: 


In [25]:
def get_correction_df(path, typos, func, minimum = True, n_neighbors=1):

    if os.path.isfile(path):
        final_df = pd.read_csv(path, index_col = 0)

    else: 
        
        final_df = typos.copy()
        rows = pd.DataFrame() 

        for typo in tqdm(typos["Typo"].tolist(), desc="Correcting Typos", total=len(typos)):

            corrections = generic_distance_correction(typo, vocab.keys(), func, minimum, n_neighbors)

            new_row = corrections.transpose().reset_index(drop=True)
            new_row.columns = [str(i) for i in range(len(new_row.columns))]
            new_row = pd.concat([new_row.iloc[0], new_row.iloc[1]], axis=0).reset_index(drop=True)
            new_row = pd.DataFrame(new_row.values.flatten()).T 
            
            row = pd.DataFrame()
            for col in new_row.columns:
                if int(col) > 4 : 
                    row[f"distance {col-n_neighbors}"] = new_row[col]
                else: 
                    row[f"correction {col}"] = new_row[col]

            rows = pd.concat([rows, row], axis=0).reset_index(drop=True)

        final_df = pd.concat([final_df, rows], axis=1)

        final_df.to_csv(path)
    
    return final_df


def get_weighted_ed_correction_df(path, typos, unigram = True, n_neighbors=1):

    if os.path.isfile(path):
        final_df = pd.read_csv(path, index_col = 0)

    else: 
        
        final_df = typos.copy()
        rows = pd.DataFrame() 

        for typo in tqdm(typos["Typo"].tolist(), desc="Correcting Typos", total=len(typos)):
            
            if unigram:
                corrections = edDistance_unigram_correction(typo, vocab.keys(), n_neighbors)
            else:
                corrections = edDistance_soundex_correction(typo, vocab.keys(), n_neighbors)

            new_row = corrections.transpose().reset_index(drop=True)
            new_row.columns = [str(i) for i in range(len(new_row.columns))]
            new_row = pd.concat([new_row.iloc[0], new_row.iloc[1]], axis=0).reset_index(drop=True)
            new_row = pd.DataFrame(new_row.values.flatten()).T 
            
            row = pd.DataFrame()
            for col in new_row.columns:
                if int(col) > 4 : 
                    row[f"distance {col-n_neighbors}"] = new_row[col]
                else: 
                    row[f"correction {col}"] = new_row[col]

            rows = pd.concat([rows, row], axis=0).reset_index(drop=True)

        final_df = pd.concat([final_df, rows], axis=1)

        final_df.to_csv(path)
    
    return final_df


def get_weighted_numeric_correction_df(path, typos, func, unigram = True, n_neighbors=1):

    if os.path.isfile(path):
        final_df = pd.read_csv(path, index_col = 0)

    else: 
        
        final_df = typos.copy()
        rows = pd.DataFrame() 

        for typo in tqdm(typos["Typo"].tolist(), desc="Correcting Typos", total=len(typos)):
            
            if unigram:
                corrections = numeric_unigram_correction(typo, vocab.keys(), func, n_neighbors)
            else:
                corrections = numeric_soundex_correction(typo, vocab.keys(), func, n_neighbors)

            new_row = corrections.transpose().reset_index(drop=True)
            new_row.columns = [str(i) for i in range(len(new_row.columns))]
            new_row = pd.concat([new_row.iloc[0], new_row.iloc[1]], axis=0).reset_index(drop=True)
            new_row = pd.DataFrame(new_row.values.flatten()).T 
            
            row = pd.DataFrame()
            for col in new_row.columns:
                if int(col) > 4 : 
                    row[f"distance {col-n_neighbors}"] = new_row[col]
                else: 
                    row[f"correction {col}"] = new_row[col]

            rows = pd.concat([rows, row], axis=0).reset_index(drop=True)

        final_df = pd.concat([final_df, rows], axis=1)

        final_df.to_csv(path)
    
    return final_df

**Jaro**

Took 9m21.8s on Louis' machine

In [27]:
jaro_correction_df = get_correction_df(DATA_PATH + "\jaro_correction_df.csv", typos, jellyfish.jaro_similarity, False, 5)
jaro_correction_df

Correcting Typos: 100%|████████████████████████████████████████████████████████████| 3183/3183 [12:14<00:00,  4.33it/s]


,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,wealthy,wealth,welty,healthy,watley,0.958333,0.916667,0.875,0.869048,0.861111
1,afford,aford,afford,axford,alford,ford,watford,0.944444,0.944444,0.944444,0.933333,0.904762
2,Catholic,CatholiaCtholic,athol,palaeolithic,pathological,alghaithi,atia,0.777778,0.766667,0.766667,0.765741,0.755556
3,cousins,coxusins,cousins,cousin,compulsions,cusi,cosiness,0.958333,0.916667,0.837121,0.833333,0.833333
4,masks,mmasks,masks,asks,mask,mass,unmasks,0.944444,0.888889,0.888889,0.888889,0.849206
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,peep,upkeep,keeps,peeps,0.933333,0.933333,0.877778,0.866667,0.866667
3179,game,gme,gme,gmes,gome,game,gm,1.0,0.916667,0.916667,0.916667,0.888889
3180,winds,wantagh,wantagh,wantage,wana,anta,want,1.0,0.904762,0.857143,0.857143,0.857143
3181,this,tsi,tsi,tsoi,tsim,gtsi,tsai,1.0,0.916667,0.916667,0.916667,0.916667


In [28]:
jaroUni_correction_df = get_weighted_numeric_correction_df(DATA_PATH + "\jaroUni_correction_df.csv", typos, jellyfish.jaro_similarity, True, 5)
jaroUni_correction_df

Correcting Typos: 100%|████████████████████████████████████████████████████████████| 3183/3183 [12:14<00:00,  4.34it/s]


,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,healthy,wealth,wealthy,watley,welty,0.869048,0.916667,0.958333,0.861111,0.875
1,afford,aford,ford,afford,watford,alford,axford,0.933333,0.944444,0.904762,0.944444,0.944444
2,Catholic,CatholiaCtholic,pathological,athol,atia,palaeolithic,alghaithi,0.766667,0.777778,0.755556,0.766667,0.765741
3,cousins,coxusins,cousin,cousins,compulsions,cosiness,cusi,0.916667,0.958333,0.837121,0.833333,0.833333
4,masks,mmasks,mass,asks,mask,masks,unmasks,0.888889,0.888889,0.888889,0.944444,0.849206
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,keeps,upkeep,peep,peeps,0.933333,0.866667,0.877778,0.933333,0.866667
3179,game,gme,game,gm,gome,gme,gmes,0.916667,0.888889,0.916667,1.0,0.916667
3180,winds,wantagh,want,wana,wantage,wantagh,anta,0.857143,0.857143,0.904762,1.0,0.857143
3181,this,tsi,tsai,tsi,gtsi,tsim,tsoi,0.916667,1.0,0.916667,0.916667,0.916667


In [29]:
jaroSdx_correction_df = get_weighted_numeric_correction_df(DATA_PATH + "\jaroUni_correction_df.csv", typos, jellyfish.jaro_similarity, False, 5)
jaroSdx_correction_df

,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,healthy,wealth,wealthy,watley,welty,0.869048,0.916667,0.958333,0.861111,0.875000
1,afford,aford,ford,afford,watford,alford,axford,0.933333,0.944444,0.904762,0.944444,0.944444
2,Catholic,CatholiaCtholic,pathological,athol,atia,palaeolithic,alghaithi,0.766667,0.777778,0.755556,0.766667,0.765741
3,cousins,coxusins,cousin,cousins,compulsions,cosiness,cusi,0.916667,0.958333,0.837121,0.833333,0.833333
4,masks,mmasks,mass,asks,mask,masks,unmasks,0.888889,0.888889,0.888889,0.944444,0.849206
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,keeps,upkeep,peep,peeps,0.933333,0.866667,0.877778,0.933333,0.866667
3179,game,gme,game,gm,gome,gme,gmes,0.916667,0.888889,0.916667,1.000000,0.916667
3180,winds,wantagh,want,wana,wantage,wantagh,anta,0.857143,0.857143,0.904762,1.000000,0.857143
3181,this,tsi,tsai,tsi,gtsi,tsim,tsoi,0.916667,1.000000,0.916667,0.916667,0.916667


**Jaro-Winkler**

Took 8m53.0s on Louis' machine

In [31]:
jw_correction_df = get_correction_df(DATA_PATH + "\jw_correction_df.csv", typos, jellyfish.jaro_winkler_similarity, False, 5)
jw_correction_df

,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,wealthy,wealth,wealthtv,weal,welty,0.975000,0.950000,0.900000,0.900000,0.900000
1,afford,aford,afford,axford,alford,ford,affords,0.955556,0.950000,0.950000,0.933333,0.923810
2,Catholic,CatholiaCtholic,athol,palaeolithic,pathological,alghaithi,atia,0.777778,0.766667,0.766667,0.765741,0.755556
3,cousins,coxusins,cousins,cousin,coxswain,compulsions,costings,0.966667,0.933333,0.883333,0.869697,0.866667
4,masks,mmasks,masks,mask,mass,asks,mma,0.950000,0.900000,0.900000,0.888889,0.883333
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,peep,kpe,keeps,kee,0.940000,0.933333,0.906667,0.880000,0.880000
3179,game,gme,gme,gmes,gome,game,gm,1.000000,0.941667,0.925000,0.925000,0.911111
3180,winds,wantagh,wantagh,wantage,want,wang,wana,1.000000,0.942857,0.914286,0.900000,0.900000
3181,this,tsi,tsi,tsim,tsoi,tsai,tsui,1.000000,0.941667,0.933333,0.933333,0.933333


In [32]:
jwUni_correction_df = get_weighted_numeric_correction_df(DATA_PATH + "\jaroUni_correction_df.csv", typos, jellyfish.jaro_winkler_similarity, True, 5)
jwUni_correction_df

,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,healthy,wealth,wealthy,watley,welty,0.869048,0.916667,0.958333,0.861111,0.875000
1,afford,aford,ford,afford,watford,alford,axford,0.933333,0.944444,0.904762,0.944444,0.944444
2,Catholic,CatholiaCtholic,pathological,athol,atia,palaeolithic,alghaithi,0.766667,0.777778,0.755556,0.766667,0.765741
3,cousins,coxusins,cousin,cousins,compulsions,cosiness,cusi,0.916667,0.958333,0.837121,0.833333,0.833333
4,masks,mmasks,mass,asks,mask,masks,unmasks,0.888889,0.888889,0.888889,0.944444,0.849206
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,keeps,upkeep,peep,peeps,0.933333,0.866667,0.877778,0.933333,0.866667
3179,game,gme,game,gm,gome,gme,gmes,0.916667,0.888889,0.916667,1.000000,0.916667
3180,winds,wantagh,want,wana,wantage,wantagh,anta,0.857143,0.857143,0.904762,1.000000,0.857143
3181,this,tsi,tsai,tsi,gtsi,tsim,tsoi,0.916667,1.000000,0.916667,0.916667,0.916667


In [33]:
jwSdx_correction_df = get_weighted_numeric_correction_df(DATA_PATH + "\jaroUni_correction_df.csv", typos, jellyfish.jaro_winkler_similarity, False, 5)
jwSdx_correction_df

,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,healthy,wealth,wealthy,watley,welty,0.869048,0.916667,0.958333,0.861111,0.875000
1,afford,aford,ford,afford,watford,alford,axford,0.933333,0.944444,0.904762,0.944444,0.944444
2,Catholic,CatholiaCtholic,pathological,athol,atia,palaeolithic,alghaithi,0.766667,0.777778,0.755556,0.766667,0.765741
3,cousins,coxusins,cousin,cousins,compulsions,cosiness,cusi,0.916667,0.958333,0.837121,0.833333,0.833333
4,masks,mmasks,mass,asks,mask,masks,unmasks,0.888889,0.888889,0.888889,0.944444,0.849206
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,keeps,upkeep,peep,peeps,0.933333,0.866667,0.877778,0.933333,0.866667
3179,game,gme,game,gm,gome,gme,gmes,0.916667,0.888889,0.916667,1.000000,0.916667
3180,winds,wantagh,want,wana,wantage,wantagh,anta,0.857143,0.857143,0.904762,1.000000,0.857143
3181,this,tsi,tsai,tsi,gtsi,tsim,tsoi,0.916667,1.000000,0.916667,0.916667,0.916667


**Edit Distance**

Took 19m10.3s on Louis' machine

In [34]:
ed_correction_df = get_correction_df(DATA_PATH + "\ed_correction_df.csv", typos, editdistance.eval, True, 5)
ed_correction_df

Correcting Typos: 100%|████████████████████████████████████████████████████████████| 3183/3183 [30:33<00:00,  1.74it/s]


,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,wealthy,wealth,healthy,healton,realogy,1,2,2,3,3
1,afford,aford,afore,ford,alford,afford,axford,1,1,1,1,1
2,Catholic,CatholiaCtholic,paleolithic,anglo-catholic,shopaholic,catholic-muslim,pathologic,7,7,7,7,7
3,cousins,coxusins,cousins,cousin,causing,mousing,coins,1,2,3,3,3
4,masks,mmasks,masks,mask,tasks,maass,umass,1,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,peep,steep,kneel,kep,1,1,2,2,2
3179,game,gme,gme,vme,gce,wme,hme,0,1,1,1,1
3180,winds,wantagh,wantagh,wantage,vantage,wattage,wastage,0,1,2,2,2
3181,this,tsi,tsi,thsi,isi,nsi,tsoi,0,1,1,1,1


In [35]:
edUni_correction_df = get_weighted_ed_correction_df(DATA_PATH + "\jaroUni_correction_df.csv", typos, True, 5)
edUni_correction_df

,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,healthy,wealth,wealthy,watley,welty,0.869048,0.916667,0.958333,0.861111,0.875000
1,afford,aford,ford,afford,watford,alford,axford,0.933333,0.944444,0.904762,0.944444,0.944444
2,Catholic,CatholiaCtholic,pathological,athol,atia,palaeolithic,alghaithi,0.766667,0.777778,0.755556,0.766667,0.765741
3,cousins,coxusins,cousin,cousins,compulsions,cosiness,cusi,0.916667,0.958333,0.837121,0.833333,0.833333
4,masks,mmasks,mass,asks,mask,masks,unmasks,0.888889,0.888889,0.888889,0.944444,0.849206
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,keeps,upkeep,peep,peeps,0.933333,0.866667,0.877778,0.933333,0.866667
3179,game,gme,game,gm,gome,gme,gmes,0.916667,0.888889,0.916667,1.000000,0.916667
3180,winds,wantagh,want,wana,wantage,wantagh,anta,0.857143,0.857143,0.904762,1.000000,0.857143
3181,this,tsi,tsai,tsi,gtsi,tsim,tsoi,0.916667,1.000000,0.916667,0.916667,0.916667


In [36]:
edSdx_correction_df = get_weighted_ed_correction_df(DATA_PATH + "\jaroUni_correction_df.csv", typos, False, 5)
edSdx_correction_df

,Word,Typo,correction 0,correction 1,correction 2,correction 3,correction 4,distance 0,distance 1,distance 2,distance 3,distance 4
0,wealthy,wealtohy,healthy,wealth,wealthy,watley,welty,0.869048,0.916667,0.958333,0.861111,0.875000
1,afford,aford,ford,afford,watford,alford,axford,0.933333,0.944444,0.904762,0.944444,0.944444
2,Catholic,CatholiaCtholic,pathological,athol,atia,palaeolithic,alghaithi,0.766667,0.777778,0.755556,0.766667,0.765741
3,cousins,coxusins,cousin,cousins,compulsions,cosiness,cusi,0.916667,0.958333,0.837121,0.833333,0.833333
4,masks,mmasks,mass,asks,mask,masks,unmasks,0.888889,0.888889,0.888889,0.944444,0.849206
...,...,...,...,...,...,...,...,...,...,...,...,...
3178,keep,kpeep,keep,keeps,upkeep,peep,peeps,0.933333,0.866667,0.877778,0.933333,0.866667
3179,game,gme,game,gm,gome,gme,gmes,0.916667,0.888889,0.916667,1.000000,0.916667
3180,winds,wantagh,want,wana,wantage,wantagh,anta,0.857143,0.857143,0.904762,1.000000,0.857143
3181,this,tsi,tsai,tsi,gtsi,tsim,tsoi,0.916667,1.000000,0.916667,0.916667,0.916667


## 1.4 Return the text file with the corrections 

### 1.4.1 Functions

In [50]:
def format_correction(typo_row):
    name = typo_row["Word"]
    typo = typo_row['Typo']
    
    #print(typo_row)
    
    neigh0 = typo_row["correction 0"]
    neigh1 = typo_row["correction 1"]
    neigh2 = typo_row["correction 2"]
    neigh3 = typo_row["correction 3"]
    neigh4 = typo_row["correction 4"]
    return f"<correction orig=\"{name}\" typo=\"{typo}\">{neigh0} {neigh1} {neigh2} {neigh3} {neigh4}</correction>"

def format_typo(typo_row):
    orig = typo_row["Word"]
    typo = typo_row['Typo']
    return f'<typo orig="{orig}">{typo}</typo>'

def replace_typos(path, typo_df, typos_str): 
 
    formatted_corr = typo_df.apply(format_correction, axis=1)
    formatted_typo = typo_df.apply(format_typo, axis=1)

    formatted_dict = dict(zip(formatted_typo, formatted_corr))

    # Erase the contents of the file if it already exists
    if os.path.isfile(path):
        os.remove(path)

    for typo_pattern in formatted_dict.keys():
        correction_pattern = formatted_dict[typo_pattern]
        typos_str = re.sub(typo_pattern, correction_pattern, typos_str)

    # Write it to a .txt file
    with open(path, "w", encoding=encoding) as out_file:
        out_file.write(typos_str)

### 1.4.2 Replace typos for all distances

In [55]:
typos_file = open(DATA_PATH + r"\typo-0.2.txt", encoding="utf-8").read()

**Jaro**

In [56]:
replace_typos(DATA_PATH + "\jaro_corrections-0.2.txt", jaro_correction_df, typos_file)

**Jaro-Winkler**

In [57]:
replace_typos(DATA_PATH + "\jw_corrections-0.2.txt", jw_correction_df, typos_file)

**Edit Distance**

In [58]:
replace_typos(DATA_PATH + "\ed_corrections-0.2.txt", ed_correction_df, typos_file)

# 2. Create evaluation metrics
Here's a couple of ideas:

- *Hard* accuracy: Does the first word match the original word?
- *Soft* accuracy: Is the original word in one of the neighbors? 

In [45]:
def evaluate_correction(path):
    corrected_file = open(path).read()
    correction_pattern = r'<correction.*?</correction>'
    matches = re.findall(correction_pattern, corrected_file)

    hardacc = []
    softacc = []
    extraction_pattern = '<correction orig="|" typo="|">|</correction>'
    for correction in matches: 
        subbed_corr = re.sub(extraction_pattern, " ", correction) # Remove all the fluff
        extracted_words = subbed_corr.split(" ")[1:-1] # remove the frst and last splits, which will always be empty

        original = extracted_words[0] 
        corrected = extracted_words[2:]

        hardacc.append(original == corrected[0])
        softacc.append(original in corrected)

    hard_accuracy = sum(hardacc) / len(hardacc)
    soft_accuracy = sum(softacc) / len(softacc)

    print(f"Hard accuracy: {round(hard_accuracy, 2)}\nSoft accuracy: {round(soft_accuracy, 2)}")
        
    return hard_accuracy, soft_accuracy

In [46]:
corr_path = DATA_PATH + r"\ed_corrections-0.2.txt"
hard, soft = evaluate_correction(corr_path)


Hard accuracy: 0.22
Soft accuracy: 0.38
